In [1]:
import polars as pl
import pandas as pd
import numpy as np
import math as math
# find a better way to import?
import sys
import os
sys.path.append(os.path.join(os.path.dirname(''), '..'))

from constants import *


In [2]:
# read all data
players = pl.read_csv('../nfl-big-data-bowl-2024/players.csv')
plays = pl.read_csv('../nfl-big-data-bowl-2024/plays.csv',infer_schema_length=100000)
games = pl.read_csv('../nfl-big-data-bowl-2024/games.csv',infer_schema_length=10000)
tracking = pl.read_csv('../nfl-big-data-bowl-2024/tracking_week*.csv',infer_schema_length=10000)

In [3]:
# normalize data
players = players.with_columns([pl.col('nflId').cast(str)])
plays = plays.join(games,on='gameId')
plays = plays.with_columns([
    (pl.col('gameId').cast(str) + '-'
     + pl.col('playId').cast(str)).alias('uniquePlayId')
])

tracking = tracking.with_columns(
    (pl.col('gameId').cast(str) + '-'
     + pl.col('playId').cast(str)).alias('uniquePlayId'),
    (pl.col('gameId').cast(str) + '-'
     + pl.col('playId').cast(str) + '-'
     + pl.col('nflId').cast(str)).alias('uniquePlayerId'),
)

# normalize position
tracking=tracking.with_columns([
    pl.when(pl.col('playDirection')=='right').then(53.3-pl.col('y')).otherwise(pl.col('y')).alias('adjustedX'),
    pl.when(pl.col('playDirection')=='right').then(pl.col('x')).otherwise(120-pl.col('x')).alias('adjustedY')
])

tracking=tracking.with_columns([
    pl.when(pl.col('event')=='ball_snap').then(pl.col('frameId')).otherwise(-1).alias('startingFrameId'),
])
tracking=tracking.with_columns([
    pl.col('startingFrameId').max().over(pl.col('uniquePlayId')),
])
tracking=tracking.with_columns([
    (pl.col('frameId') - pl.col('startingFrameId')).alias('framesSinceSnap'),
])

# normalize orientation 'o' and direction 'dir'
# convert 'NA' to 0
replacement_values = {'NA': '0'}
tracking = tracking.with_columns(
    pl.col('o').apply(lambda x: replacement_values.get(x, x)),
    pl.col('dir').apply(lambda x: replacement_values.get(x, x)),
)

tracking=tracking.with_columns([
    pl.when(pl.col('playDirection')=='right').then(pl.col('dir').cast(pl.Float64)).otherwise(180-pl.col('dir').cast(pl.Float64)).alias('adjustedDir'),
    pl.when(pl.col('playDirection')=='right').then(pl.col('o').cast(pl.Float64)).otherwise((180+pl.col('o').cast(pl.Float64))%360).alias('adjustedO'),
])

In [55]:
CONE_ANGLE = 20 # degrees
MAX_DISTANCE = 5 # yards
BLOCKING_RADIUS = 1 # yards

# row = [o, dir, adjustedX, adjustedY, oDefender, dirDefender, adjustedXDefender, adjustedYDefender]
def looking_to_block_or_blocking_df_fn(row) -> int:
    blocking_status = 0
    player1 = row[0:4]
    player2 = row[4:]

    if is_in_vision_cone(player1, player2):
        blocking_status = 1

    if is_blocking(player1, player2):
        blocking_status = 2

    return blocking_status
    

def looking_to_block_or_blocking(player1: tuple, player2: tuple) -> int:
    if is_in_vision_cone(player1, player2):
        if is_blocking(player1, player2):
            return 2
            
        return 1

    return 0

def is_in_vision_cone(player1: tuple, player2: tuple) -> bool:
    return (is_in_angle(player1, player2) and is_in_distance(player1, player2))

def is_in_angle(player1: tuple, player2: tuple) -> bool:
    half_cone_angle = CONE_ANGLE / 2

    y_dist = player2[3] - player1[3]
    x_dist = player2[2] - player1[2]
    player1_orientation = float(player1[0]) if type(player1[0]) == str else player1[0]

    # print("isinangle")
    # print("player1")
    # print(player1)

    # print("player2")
    # print(player2)

    # print("player1_ori")
    # print(player1_orientation)

    angle = math.degrees(math.atan2(y_dist, x_dist))

    # print("angle")
    # print(angle)

    if player1_orientation - half_cone_angle <= angle <= player1_orientation + half_cone_angle:
        return True
    
    return False

def is_in_distance(player1: tuple, player2: tuple) -> bool:
    distance_between_players = calculate_distance(player1, player2)
    if distance_between_players <= MAX_DISTANCE:
        return True
    
    return False

def is_blocking(player1: tuple, player2: tuple) -> bool:
    distance_between_players = calculate_distance(player1, player2)
    if distance_between_players <= BLOCKING_RADIUS:
        return True

    return False


def calculate_distance(player1: tuple, player2: tuple) -> float:
    y_dist = abs(player1[3] - player2[3])
    x_dist = abs(player1[2] - player2[2])

    # print(y_dist)
    # print(x_dist)
    # print(math.sqrt(x_dist**2 + y_dist**2))

    return math.sqrt(x_dist**2 + y_dist**2)

In [5]:
# This is just for testing - when we go for it we'll want to let it go against everything
labeled = tracking.filter(pl.col('gameId')==2022091104)


In [6]:
players = labeled.join(players,on='nflId',how='left')
players = players.join(games.select(['gameId','homeTeamAbbr','visitorTeamAbbr']),on='gameId')
players = players.with_columns([
    pl.when(pl.col('club')==pl.col('homeTeamAbbr'))
    .then(pl.col('visitorTeamAbbr'))
    .otherwise(pl.col('homeTeamAbbr'))
    .alias('opponentClub')
])


In [7]:
players = players.join(
    players,
    left_on=['gameId','playId','frameId','club'],
    right_on=['gameId','playId','frameId','opponentClub'],
    suffix='Defender'
)

In [8]:
blocking_df = players.select(
    'adjustedO', 'adjustedDir', 'adjustedX', 'adjustedY', 'adjustedODefender', 'adjustedDirDefender', 'adjustedXDefender', 'adjustedYDefender'
).apply(looking_to_block_or_blocking_df_fn)

In [9]:
blocking_df = blocking_df.to_series()

In [10]:
players = players.with_columns(blockType=blocking_df)

In [11]:
test = pl.read_parquet('../labeledDataAttempt/labeledBlocks.parquet')

In [12]:
blocking_df = players.select(
    'adjustedO', 'adjustedDir', 'adjustedX', 'adjustedY', 'adjustedODefender', 'adjustedDirDefender', 'adjustedXDefender', 'adjustedYDefender'
).apply(looking_to_block_or_blocking_df_fn)

blocking_df = blocking_df.to_series()

players = players.with_columns(blockType=blocking_df)

In [13]:
test.filter(pl.col('displayName')=="Frank Ragnow").filter(pl.col('displayNameDefender')=="Kyzir White").filter(pl.col('gameId')==2022091104).select(pl.col('adjustedX'), pl.col('adjustedY'), pl.col('adjustedXDefender'), pl.col('adjustedYDefender'), pl.col('blockType'), pl.col('framesSinceSnap'), pl.col('playId')).filter(pl.col('framesSinceSnap')==3)

adjustedX,adjustedY,adjustedXDefender,adjustedYDefender,blockType,framesSinceSnap,playId
f64,f64,f64,f64,i32,i64,i64
23.38,33.52,21.43,38.93,1,3,86
23.99,83.42,24.94,88.3,0,3,107


In [14]:
players.filter(pl.col('displayName')=="Frank Ragnow").filter(pl.col('displayNameDefender')=="Kyzir White").filter(pl.col('gameId')==2022091104).select(pl.col('adjustedX'), pl.col('adjustedY'), pl.col('adjustedXDefender'), pl.col('adjustedYDefender'), pl.col('blockType'), pl.col('framesSinceSnap'), pl.col('playId')).filter(pl.col('framesSinceSnap')==3).filter(pl.col('playId')==86)

adjustedX,adjustedY,adjustedXDefender,adjustedYDefender,blockType,framesSinceSnap,playId
f64,f64,f64,f64,i64,i64,i64
23.38,33.52,21.43,38.93,0,3,86


In [56]:
test_data = players.filter(pl.col('jerseyNumber')=='88').filter(pl.col('gameId')==2022091104).filter(pl.col('playId')==86)
blocking_df = test_data.select(
    'adjustedO', 'adjustedDir', 'adjustedX', 'adjustedY', 'adjustedODefender', 'adjustedDirDefender', 'adjustedXDefender', 'adjustedYDefender'
).apply(looking_to_block_or_blocking_df_fn)

blocking_df = blocking_df.to_series()

test_data = test_data.with_columns(blockType=blocking_df)

In [57]:
pl.Config(tbl_rows=200)
test_data.select(pl.col('jerseyNumberDefender'), pl.col('framesSinceSnap'), pl.col('blockType')).filter(pl.col('blockType')!=0)

jerseyNumberDefender,framesSinceSnap,blockType
str,i64,i64
"""97""",12,2
"""97""",13,2
"""97""",14,2
"""97""",15,2
"""97""",16,2
"""97""",17,2
"""97""",18,2
"""97""",19,2
"""97""",20,2
